# Data Extractor

In [1]:
# TODO: Use the json secrets for initialize earth engine (json key for service account)
# service_account = 'tec-semester@plantgrowthcycle.iam.gserviceaccount.com'
# credentials = ee.ServiceAccountCredentials(service_account, 'plantgrowthcycle-8e4112c2f80a.json')
# ee.Initialize(credentials)

In [2]:
import ee
ee.Initialize()
ee.Authenticate()
import geetools
import geemap
import os

Enter verification code: 4/1ARtbsJow5byirbSgB8hR1j-vSLAEKe-t7I8M-eCyAnR0TFc7W1gKBjmTxV4

Successfully saved authorization token.


In [3]:
#Angel se la come

class DataExtractor():
    
    def __init__(self, data_collection, start_date, end_date, bands):
        self.data_collection = data_collection
        self.start_date = start_date
        self.end_date = end_date
        self.bands = bands
        self.point = None
        self.region = None
        self.image_collection = None
        self.Map = None
 
    def set_point(self, point):
        self.point = point
        
    def set_interest_region(self, meters):
        """ 
        Select an arbitrary point and a distance in (meters),
        to construct a rectangle centered on the given point.
        """
        point = ee.Geometry.Point(self.point)        
        self.region = point.buffer(meters).bounds()
        
        Map = geemap.Map()
        Map.setCenter(self.point[0], self.point[1])
        Map.addLayer(point,
             {'color': 'black'},
             'Geometry [black]: point')
        
        Map.addLayer(self.region)
        
        self.Map = Map
        
        return self.Map

    def get_data_visualization(self):
        """ Get the Earth Engine image collection with the desired characteristics """
        self.image_collection = (
            ee.ImageCollection(self.data_collection)
            .select(self.bands)
            .filterBounds(self.region)
            .filterDate(self.start_date, self.end_date)
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
        )
        
        print(f'The size of the collection is: {self.image_collection.size().getInfo()}')

        visualization = {
            "bands" : self.bands,
            "min": 300, 
            "max": 3500
        }

        self.Map.addLayer(
            ee_object=self.image_collection, 
            vis_params=visualization,
            name="Sentinel-2",
            shown=False)

        self.Map.add_time_slider(
            ee_object=self.image_collection,
            vis_params=visualization,
            time_interval=5
        )
    
    def extract_data(self):
        """ Iterates over the ImageCollection and send the images to Drive """        
        for band in self.bands:
            extra = dict(sat="S-HARMONIZED", band=band)
            geetools.batch.Export.imagecollection.toDrive(
                collection=self.image_collection.select(band),
                region=self.region,
                namePattern="{sat}_{id}_{system_date}_{band}",
                datePattern = "ddMMMy",
                dataType="int",
                folder="ClimateData_test",
                extra=extra,
                verbose=True
            )

In [4]:
# Parameters.
data_collection = "COPERNICUS/S2_SR_HARMONIZED"
# Time windows.
start_date = "2021-08-01"
# start_date = "2020-04-15"
end_date = "2021-09-01"
# end_date = "2020-11-01"
# Color bands.
bands =  ["B4","B3","B2"] # RGB channels.
# Coordinates
coordinates = {
    "Germany" : {
        "0" : [10.109178, 52.295949],
    } ,
    "Iowa" : {
        "0" : [-91.47321072391104, 41.87728596488476],
    }
}

In [5]:
# Call the DataExtractorClass.
D = DataExtractor(
    data_collection=data_collection,
    start_date=start_date,
    end_date=end_date,
    bands=bands
)

D.set_point(coordinates["Germany"]["0"])
D.set_interest_region(2000)

Map(center=[52.295949, 10.109178], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [6]:
D.get_data_visualization()

The size of the collection is: 3


In [7]:
D.extract_data()

exporting S-HARMONIZED_20210812T102559_20210812T103532_T32UND_12Aug2021_B4 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210814T102031_20210814T102609_T32UNC_14Aug2021_B4 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210814T102031_20210814T102609_T32UND_14Aug2021_B4 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210812T102559_20210812T103532_T32UND_12Aug2021_B3 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210814T102031_20210814T102609_T32UNC_14Aug2021_B3 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210814T102031_20210814T102609_T32UND_14Aug2021_B3 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210812T102559_20210812T103532_T32UND_12Aug2021_B2 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210814T102031_20210814T102609_T32UNC_14Aug2021_B2 to folder 'ClimateData_test' in GDrive
exporting S-HARMONIZED_20210814T102031_20210814T102609_T32UND_14Aug2021_B2 to folder 'Cl